In [ ]:
def find_ngrams(input_list, n):
    return zip(*[input_list[i:] for i in range(n)])

word_dict = find_ngrams(text_content,3)

#for i in range(listLen):
#    word_dict['_'.join(scoredList[i][0])] = scoredList[i][1]

wordCloud.generate(" ".join(word_dict)) 
plt.title('Most frequently occurring trigrams connected with an underscore_')
plt.imshow(wordCloud)
plt.axis("off")
plt.show()

### Unsupervised Learning: As only the job titles are available, I am trying to cluster the titles based on string similarity

In [ ]:
from nltk.metrics.distance import jaccard_distance
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt


pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
])        
X = pipeline.fit_transform(clean_jobtitle_list).todense()

pca = PCA(n_components=5).fit(X)
data2D = pca.transform(X)
plt.scatter(data2D[:,0], data2D[:,1])

kmeans = KMeans(n_clusters=5).fit(X)
centers2D = pca.transform(kmeans.cluster_centers_)

plt.hold(True)
plt.scatter(centers2D[:,0], centers2D[:,1], 
            marker='x', s=200, linewidths=3, c='r')
plt.show()

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(clean_jobtitle_list)

#print(X)
true_k = 10
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print ("Cluster %d:" % i)
    for ind in order_centroids[i, :10]:
        print (terms[ind])

## Word2Vec based clustering and function recommendation
### need labelled dataset

In [ ]:
import gensim

from gensim.models import Word2Vec

# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)  



def avg_sentence(sentence, wv):
    v = np.zeros(300)
    for w in sentence:
        if w in wv:
            v += wv[w]
    return v / len(sentence)

def cosine_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))



with open('First100.csv') as csvfile:
    rows = csv.reader(csvfile)
    res = list(zip(rows))

data=res[1:-1]
data

inputv = avg_sentence("Marketing Partner".split(), model.wv)
avgs = list(map(lambda p: p + (avg_sentence(p[0][0].split(), model.wv),), data))
sims = list(map(lambda p: p[0][:2] + (cosine_sim(inputv, p[0][2]),), avgs))

most_similar = sorted(sims, key=lambda p: p[2], reverse=True)

print(most_similar)

#title_list=[list(chain.from_iterable(clean_title)) for clean_title in clean_jobtitle_list]
#print(title_list[:100])